In [1]:
!pip install --quiet snscrape
!pip install -U --quiet scikit-learn
!pip install --quiet pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 2.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import random
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import snscrape.modules.twitter as sntwitter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# Tentukan keyword dan rentang tanggal pencarian
keyword = "blackpink"
start_date = "2020-01-01"
end_date = "2022-12-31"

# Buat query pencarian
query = f"{keyword} since:{start_date} until:{end_date} lang:en"

# Scraping tweet dengan snscrape
tweets = []
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    tweets.append([tweet.date, tweet.rawContent, tweet.user.username, tweet.likeCount, tweet.replyCount, tweet.retweetCount])
    if i >= 10000: # Mengambil data hingga 10.000 tweet
        break

# Random 2000 data dari hasil scraping
if len(tweets) > 2000:
    tweets = random.sample(tweets, 2000)

# Simpan hasil scraping ke dalam dataframe
df = pd.DataFrame(tweets, columns=["Tanggal", "Tweet", "Username", "Likes", "Replies", "Retweets"])

In [4]:
# Menyimpan dataset dengan kolom sentimen baru
df.to_csv("data.csv", index=False)

In [44]:
df = pd.read_csv('data.csv')

In [52]:
df.tail()

,Tanggal,Tweet,Username,Likes,Replies,Retweets,Sentimen,Sentimen Label
1995,2022-12-30 14:12:17+00:00,hisfavnana marbronzo sorry let go blackpink,hwyfrz,2,0,0,-0.5,negatif
1996,2022-12-30 15:11:03+00:00,im blackpink sorry dont support group pink bla...,slaypinks04,2,0,1,-0.3,negatif
1997,2022-12-30 18:15:58+00:00,d0r0r09 blackpink hello everyone name roséhh,BORNPINKxbp,0,0,0,0.0,netral
1998,2022-12-30 23:23:17+00:00,avatricesource fav music blackpink savewarrior...,sounder57324414,0,0,0,0.0,netral
1999,2022-12-30 22:51:22+00:00,kchartsmaster _iuofficial limyoungwoong bts_tw...,HYCU107,12,0,0,0.0,netral


In [40]:
df[['Tanggal', 'Tweet']]

,Tanggal,Tweet
0,2022-12-30 11:51:37+00:00,.@BLACKPINK's 'Shut Down' has surpassed 2.4 mi...
1,2022-12-30 08:21:52+00:00,@mylenexxxxxxx @BBU_BLACKPINK @BLACKPINK Yes. ...
2,2022-12-30 23:30:43+00:00,@quiuzlui @girlbossivestan @Seulmin26 @kcharts...
3,2022-12-30 23:26:25+00:00,@elena_drye Ok good I was kinda worried tbh.
4,2022-12-30 09:44:54+00:00,#RishabhPant #HeerabenModi Om Shanti #ThunivuC...
...,...,...
1995,2022-12-30 14:12:17+00:00,"@hisfavnana @marbronzo Sorry, let's go with bl..."
1996,2022-12-30 15:11:03+00:00,Im here only for blackpink sorry. I don't supp...
1997,2022-12-30 18:15:58+00:00,@D0r0r09 @BLACKPINK Hello everyone my name is ...
1998,2022-12-30 23:23:17+00:00,@avatricesource Fav music #BLACKPINK #SaveWarr...


In [46]:
# Fungsi untuk melakukan preprocessing
def preprocess_tweet(tweet):
    # Menghapus tanda baca dan karakter khusus
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Mengubah ke huruf kecil
    tweet = tweet.lower()

    # Menghapus stopwords
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(tweet)
    tokens = [token for token in tokens if token not in stop_words]
    
    # Melakukan lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Menggabungkan kembali token menjadi kalimat
    tweet = ' '.join(tokens)
    return tweet

# Preprocessing pada kolom Tweet dalam DataFrame
df['Tweet'] = df['Tweet'].apply(preprocess_tweet)

In [47]:
# Cek sentimen dengan TextBlob dan tambahkan kolom Sentimen
df['Sentimen'] = df['Tweet'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)

# Tambahkan kolom Sentimen Label berdasarkan nilai Sentimen
df['Sentimen Label'] = df['Sentimen'].apply(lambda sentimen: "positif" if sentimen > 0 else "negatif" if sentimen < 0 else "netral")

In [60]:
# Menjumlahkan sentimen
jumlah_sentimen = df['Sentimen Label'].value_counts()

# Membuat dataframe baru
df_sentimen = pd.DataFrame(jumlah_sentimen).reset_index()

# Mengubah nama kolom
df_sentimen.columns = ['Sentimen', 'Jumlah']

# Menampilkan hasil
df_sentimen

,Sentimen,Jumlah
0,netral,941
1,positif,715
2,negatif,715


In [65]:
# Hitung jumlah data untuk setiap Sentimen Label
sentiment_counts = df['Sentimen Label'].value_counts()
neutral_count = sentiment_counts['netral']
positive_count = sentiment_counts['positif']
negative_count = sentiment_counts['negatif']

difference = abs(positive_count - neutral_count)

# Cek apakah ada selisih data dengan proporsi yang diinginkan
print("Selisih data:", difference)

# Tambahkan data dengan sentimen negatif secara otomatis
new_tweets = []
for i in range(difference):
    tweet = sntwitter.TwitterSearchScraper(query).get_items()
    new_tweet = {'Tanggal': tweet.date, 'Tweet': tweet.rawContent, 'Username': tweet.user.username, 'Likes': tweet.likeCount, 'Replies': tweet.replyCount, 'Retweets': tweet.retweetCount, 'Sentimen': TextBlob(preprocess_tweet(tweet.rawContent)).sentiment.polarity, 'Sentimen Label': 'negatif'}
    new_tweets.append(new_tweet)

# # Tambahkan data dengan sentimen positif secara otomatis
# new_tweets = []
# for i in range(difference):
#     tweet = sntwitter.TwitterSearchScraper(query).get_items()
#     new_tweet = {'Tanggal': tweet.date, 'Tweet': tweet.rawContent, 'Username': tweet.user.username, 'Likes': tweet.likeCount, 'Replies': tweet.replyCount, 'Retweets': tweet.retweetCount, 'Sentimen': TextBlob(preprocess_tweet(tweet.rawContent)).sentiment.polarity, 'Sentimen Label': 'positif'}
#     new_tweets.append(new_tweet)

# Gabungkan dataframe yang baru dengan dataframe yang telah ada
new_df = pd.DataFrame(new_tweets)
df = pd.concat([df, new_df], ignore_index=True)

Selisih data: 226


In [66]:
# Menjumlahkan sentimen
jumlah_sentimen = df['Sentimen Label'].value_counts()

# Membuat dataframe baru
df_sentimen = pd.DataFrame(jumlah_sentimen).reset_index()

# Mengubah nama kolom
df_sentimen.columns = ['Sentimen', 'Jumlah']

# Menampilkan hasil
df_sentimen

,Sentimen,Jumlah
0,positif,941
1,netral,941
2,negatif,941


In [68]:
df.to_csv("data_sentimen.csv", index=False)
df.head()

,Tanggal,Tweet,Username,Likes,Replies,Retweets,Sentimen,Sentimen Label
0,2022-12-30 11:51:37+00:00,blackpinks shut surpassed 24 million unique li...,chartsblackpink,961.0,0.0,176.0,0.375,positif
1,2022-12-30 08:21:52+00:00,mylenexxxxxxx bbu_blackpink blackpink yes lol ...,YamanakaBitch,0.0,0.0,0.0,0.800,positif
2,2022-12-30 23:30:43+00:00,quiuzlui girlbossivestan seulmin26 kchartsmast...,ramadhaniWy,2.0,1.0,0.0,0.200,positif
3,2022-12-30 23:26:25+00:00,elena_drye ok good kinda worried tbh,born_blackpink,0.0,1.0,0.0,0.600,positif
4,2022-12-30 09:44:54+00:00,rishabhpant heerabenmodi om shanti thunivuchar...,theusaprint,1.0,0.0,3.0,0.000,netral


In [69]:
# Membuat vektor fitur menggunakan CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Tweet'])

# Menentukan target
y = df['Sentimen Label']

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat model Naive Bayes Classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Melakukan prediksi pada data uji
y_pred = clf.predict(X_test)

# Menghitung akurasi
acc = accuracy_score(y_test, y_pred)
print("Akurasi:", acc)

Akurasi: 0.7115044247787611


In [70]:
# Membuat model SVM
svm_model = SVC(kernel='linear', probability=True)

# Membuat model Naive Bayes
nb_model = MultinomialNB()

# Menggabungkan kedua model dalam voting classifier
voting_model = VotingClassifier(estimators=[('svm', svm_model), ('nb', nb_model)], voting='soft')

# Melatih model pada data training
voting_model.fit(X_train, y_train)

VotingClassifier(estimators=[('svm', SVC(kernel='linear', probability=True)),
                             ('nb', MultinomialNB())],
                 voting='soft')

In [71]:
# Melakukan prediksi sentimen pada data testing
y_pred = voting_model.predict(X_test)

# Menghitung akurasi prediksi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi prediksi: {accuracy}")

Akurasi prediksi: 0.7433628318584071


In [73]:
# Prediksi sentimen pada data baru
new_tweet = "BLACKPINK's Jisoo, Jennie, Rosé and Lisa share a mix of memories, nerves, excitement and anticipation as they prepare to close out day 2 of Coachella 2023."
new_tweet = preprocess_tweet(new_tweet)
X_test = vectorizer.transform([new_tweet])
y_pred = voting_model.predict(X_test)

# Cetak hasil prediksi
print("Hasil prediksi sentimen:")
print("Sentimen: ", y_pred[0])

Hasil prediksi sentimen:
Sentimen:  netral
